In [1]:
import serial
import socket
import time
import json
import csv
import os
import numpy as np

# Serial and socket setup
PRINTER_PORT = 'COM15'
SENSOR_PORT = 'COM6'
BAUD_PRINTER = 250000
BAUD_SENSOR = 57600
SOCKET_HOST = '172.26.128.172'
SOCKET_PORT = 65432
CSV_FILENAME = 'sensor_data.csv'

with open(CSV_FILENAME, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['x', 'z', 'strain_0', 'strain_1', 'strain_2', 'strain_3', 'strain_4', 'force'])

# Initialize serial ports
ser = serial.Serial(PRINTER_PORT, BAUD_PRINTER, timeout=2)
sensor = serial.Serial(SENSOR_PORT, BAUD_SENSOR, timeout=2)
time.sleep(3)

baseline = None

def flush_startup():
    ser.reset_input_buffer()
    time.sleep(1)
    while ser.in_waiting:
        line = ser.readline().decode(errors='ignore').strip()
        if line:
            print("[Printer]", line)

def flush_sensor():
    sensor.reset_input_buffer()
    time.sleep(1)
    print("Waiting for sensor to stabilize...")
    timeout = time.time() + 3
    while time.time() < timeout:
        if sensor.in_waiting:
            line = sensor.readline().decode(errors='ignore').strip()
            if line:
                print("[Sensor boot]", line)

def send_gcode(command):
    print(f">> {command}")
    ser.write((command + '\n').encode())
    while True:
        line = ser.readline().decode(errors='ignore').strip()
        if line == "ok":
            break
        elif line:
            print(line)

def wait_until_idle():
    send_gcode("M400")

def move_to(x=None, z=None, feedrate=300):
    cmd = "G1"
    if x is not None:
        cmd += f" X{x}"
    if z is not None:
        cmd += f" Z{z}"
    cmd += f" F{feedrate}"
    send_gcode(cmd)
    wait_until_idle()

def read_sensor():
    print("Reading sensor values...")
    sensor.reset_input_buffer()
    sensor.write(b'b')
    time.sleep(0.5)

    readings = []
    timeout = time.time() + 2
    while time.time() < timeout:
        if sensor.in_waiting:
            line = sensor.readline().decode(errors='ignore').strip()
            if line:
                try:
                    val = float(line)
                    readings.append(val)
                    print(f"  Raw: {val:.3f}")
                except:
                    print(f"Could not parse: {line}")
    
    print(f"Total readings collected: {len(readings)}")
    return readings

def read_baseline():
    print("Capturing sensor baseline...")
    readings = read_sensor()
    if readings:
        print(f"Baseline captured: {len(readings)} sensors")
        for i, val in enumerate(readings):
            print(f"   Sensor {i}: {val:.3f}")
    return readings

def calculate_and_display_differences(raw_readings, baseline, position_info=""):
    if len(raw_readings) != len(baseline):
        print(f"Sensor count mismatch: got {len(raw_readings)}, expected {len(baseline)}")
        return None, None
    
    all_differences = [-(raw_readings[i] - baseline[i]) * 1000 for i in range(len(raw_readings)-1)]
    
    strain_differences = all_differences[:5]
    force_difference = raw_readings[5] * 0.098 if len(raw_readings) > 5 else 0.0

    print(f"Sensor differences{position_info}:")
    for i in range(5):
        if i < len(raw_readings):
            raw = raw_readings[i]
            base = baseline[i]
            diff = strain_differences[i]
            print(f"   Strain Sensor {i}: {raw:.3f} - {base:.3f} = {diff:+7.1f} Pa")

    if len(raw_readings) > 5:
        raw_force = raw_readings[5]
        base_force = baseline[5]
        print(f"   Force Sensor 5:   {raw_force:.3f} - {base_force:.3f} = {force_difference:+7.1f} N")

    if strain_differences:
        max_strain_diff = max(strain_differences, key=abs)
        max_strain_idx = strain_differences.index(max_strain_diff)
        avg_abs_strain_diff = sum(abs(d) for d in strain_differences) / len(strain_differences)
        
        print(f"Max strain difference: {max_strain_diff:+7.1f} Pa (Sensor {max_strain_idx})")
        print(f"Avg abs strain difference: {avg_abs_strain_diff:6.1f} Pa")
        print(f"Force difference: {force_difference:+7.1f} N")
    
    return strain_differences, force_difference

def save_to_csv(x, z, strain_diffs, force):
    with open(CSV_FILENAME, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([x, z] + strain_diffs + [force])

flush_startup()
flush_sensor()

send_gcode("G21")
send_gcode("G90")
send_gcode("M211 S0")

baseline = read_baseline()
if not baseline:
    print("Failed to read baseline sensor values.")
    ser.close()
    sensor.close()
    exit()

try:
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.connect((SOCKET_HOST, SOCKET_PORT))
    print("Connected to WSL-side MLP server.")
except Exception as e:
    print(f"Failed to connect to server: {e}")
    ser.close()
    sensor.close()
    exit()

print("\n=== Indenter Control Interface ===")
print("Commands:")
print("  move x z           -> Move to X/Z position + read + send sensor data")
print("  origin             -> Set current position as (0,0,0)")
print("  indent x0 x1 step z_down z_up -> Automated loop")
print("  retare             -> Re-capture baseline readings")
print("  test               -> Test sensor reading without moving")
print("  gcode YOUR_COMMAND -> Send custom G-code")
print("  exit               -> Quit")

try:
    while True:
        cmd = input("\n>> ").strip()
        if not cmd:
            continue
        parts = cmd.split()

        if parts[0] == "move" and len(parts) == 3:
            x = float(parts[1])
            z = float(parts[2])
            print(f"Moving to X={x}, Z={z}")
            move_to(x=x, z=z)
            raw = read_sensor()
            strain_diffs, force_diff = calculate_and_display_differences(raw, baseline, f" at X={x}, Z={z}")
            if strain_diffs is not None:
                payload_data = strain_diffs + [force_diff]
                payload = {"x": x, "z": z, "readings": payload_data}
                try:
                    sock.sendall(json.dumps(payload).encode() + b"\n")
                    print("Sent data to server")
                except Exception as e:
                    print(f"Failed to send data: {e}")
                save_to_csv(x, z, strain_diffs, force_diff)

        elif parts[0] == "test":
            print("Testing sensor reading...")
            raw = read_sensor()
            strain_diffs, force_diff = calculate_and_display_differences(raw, baseline, " (test)")
            if strain_diffs is not None:
                save_to_csv(0, 0, strain_diffs, force_diff)

        elif parts[0] == "retare":
            print("Re-capturing baseline...")
            baseline = read_baseline()
            if not baseline:
                print("Failed to re-capture baseline!")

        elif parts[0] == "origin":
            send_gcode("G92 X0 Y0 Z0")
            print("Origin set to (0,0,0).")

        elif parts[0] == "indent" and len(parts) == 6:
            x0 = float(parts[1])
            x1 = float(parts[2])
            step = float(parts[3])
            z_down = float(parts[4])
            z_up = float(parts[5])

            print(f"Starting indent sequence: X={x0} to {x1} (step {step}), Z_down={z_down}, Z_up={z_up}")
            for x in np.arange(x0, x1 + step/2, step):
                print(f"\nIndenting at X={x}")
                move_to(x=x, z=z_up)
                time.sleep(0.3)
                move_to(z=z_down)
                time.sleep(0.2)
                raw = read_sensor()
                strain_diffs, force_diff = calculate_and_display_differences(raw, baseline, f" at X={x}, Z={z_down}")
                if strain_diffs is not None:
                    payload_data = strain_diffs + [force_diff]
                    payload = {"x": x, "z": z_down, "readings": payload_data}
                    try:
                        sock.sendall(json.dumps(payload).encode() + b"\n")
                        print("Sent indent data")
                    except Exception as e:
                        print(f"Failed to send data: {e}")
                    save_to_csv(x, z_down, strain_diffs, force_diff)
                time.sleep(0.3)
                move_to(z=z_up)
                time.sleep(0.3)

        elif parts[0] == "gcode" and len(parts) > 1:
            gcode_cmd = ' '.join(parts[1:])
            send_gcode(gcode_cmd)

        elif parts[0] == "exit":
            print("Exiting...")
            break

        else:
            print("Invalid command. Type 'exit' to quit.")

except KeyboardInterrupt:
    print("\nInterrupted by user")
except Exception as e:
    print(f"Unexpected error: {e}")
    import traceback
    traceback.print_exc()

finally:
    print("Cleaning up...")
    try:
        ser.close()
        sensor.close()
        sock.close()
    except:
        pass
    print("Cleanup complete")


Waiting for sensor to stabilize...
[Sensor boot] Sensor 2 OK
[Sensor boot] Sensor 3 OK
[Sensor boot] Sensor 4 OK
>> G21
echo:SD card ok
>> G90
>> M211 S0
Capturing sensor baseline (tare)...
Reading sensor values...
  Raw: 95.002
  Raw: 98.849
  Raw: 99.384
  Raw: 91.912
  Raw: 67.097
  Raw: 0.186
Total readings collected: 6
Baseline captured: 6 sensors
   Sensor 0: 95.002
   Sensor 1: 98.849
   Sensor 2: 99.384
   Sensor 3: 91.912
   Sensor 4: 67.097
   Sensor 5: 0.186
Connected to WSL-side MLP server.

=== Indenter Control Interface ===
Commands:
  move x z           -> Move to X/Z position + read + send sensor data
  origin             -> Set current position as (0,0,0)
  indent x0 x1 step z_down z_up -> Automated loop
  retare             -> Re-capture baseline readings
  test               -> Test sensor reading without moving
  gcode YOUR_COMMAND -> Send custom G-code
  exit               -> Quit



>>  origin


>> G92 X0 Y0 Z0
echo:endstops hit:  Z:0.00
Origin set to (0,0,0).



>>  move 0 -1


Moving to X=0.0, Z=-1.0
>> G1 X0.0 Z-1.0 F300
>> M400
Reading sensor values...
  Raw: 96.760
  Raw: 101.339
  Raw: 102.809
  Raw: 94.427
  Raw: 68.489
  Raw: 269.955
Total readings collected: 6
Sensor differences at X=0.0, Z=-1.0:
   Strain Sensor 0: 96.760 - 95.002 = -1757.9 Pa
   Strain Sensor 1: 101.339 - 98.849 = -2489.8 Pa
   Strain Sensor 2: 102.809 - 99.384 = -3424.8 Pa
   Strain Sensor 3: 94.427 - 91.912 = -2514.6 Pa
   Strain Sensor 4: 68.489 - 67.097 = -1392.3 Pa
   Force Sensor 5:   269.955 - 0.186 =   +26.5 N
Max strain difference: -3424.8 Pa (Sensor 2)
Avg abs strain difference: 2315.9 Pa
Force difference:   +26.5 N
Sent data to server


...

Moving to X=0.0, Z=-3.25
>> G1 X0.0 Z-3.25 F300
>> M400
Reading sensor values...
  Raw: 98.782
  Raw: 107.394
  Raw: 111.599
  Raw: 100.406
  Raw: 70.376
  Raw: 833.516
Total readings collected: 6
Sensor differences at X=0.0, Z=-3.25:
   Strain Sensor 0: 98.782 - 95.002 = -3780.1 Pa
   Strain Sensor 1: 107.394 - 98.849 = -8545.0 Pa
   Strain Sensor 2: 111.599 - 99.384 = -12215.2 Pa
   Strain Sensor 3: 100.406 - 91.912 = -8493.6 Pa
   Strain Sensor 4: 70.376 - 67.097 = -3278.9 Pa
   Force Sensor 5:   833.516 - 0.186 =   +81.7 N
Max strain difference: -12215.2 Pa (Sensor 2)
Avg abs strain difference: 7262.6 Pa
Force difference:   +81.7 N
Sent data to server



>>  move 0 -3.5


Moving to X=0.0, Z=-3.5
>> G1 X0.0 Z-3.5 F300
>> M400
Reading sensor values...
  Raw: 99.500
  Raw: 107.888
  Raw: 111.427
  Raw: 100.804
  Raw: 70.532
  Raw: 875.089
Total readings collected: 6
Sensor differences at X=0.0, Z=-3.5:
   Strain Sensor 0: 99.500 - 95.002 = -4497.9 Pa
   Strain Sensor 1: 107.888 - 98.849 = -9038.1 Pa
   Strain Sensor 2: 111.427 - 99.384 = -12043.4 Pa
   Strain Sensor 3: 100.804 - 91.912 = -8892.0 Pa
   Strain Sensor 4: 70.532 - 67.097 = -3435.4 Pa
   Force Sensor 5:   875.089 - 0.186 =   +85.8 N
Max strain difference: -12043.4 Pa (Sensor 2)
Avg abs strain difference: 7581.4 Pa
Force difference:   +85.8 N
Sent data to server



>>  exit


Exiting...
Cleaning up...
Cleanup complete
